<a href="https://colab.research.google.com/github/sshin1179/Deep-Learning-Study/blob/main/JX_Partners_Sample_Note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JX Partners Project: 기업의 상장 후 주가 움직임 분석
- 2023년 1월-12월 사이에 상장한 기업들의 주가 움직임을 분석
- 데이터는 [KRX 정보데이터시스템 -> IPO 관련 통계 -> 공모가 대비 등락률](http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC02020103)에서 활용

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import timedelta

def prepare(start="2023-01-01", end="2023-12-31"):
    from google.colab import files

    # Upload and load the Excel file containing stock data
    uploaded = files.upload()
    df = pd.read_excel('practice.xlsx')

    # Filter out SPAC stocks
    df = df[~df['종목명'].str.contains('스팩')]

    # Format stock codes and create a dictionary of stock names and codes
    stock_info = df[['종목명', '종목코드', '공모가(D)', '상장일_종가']].copy()
    stock_info['종목코드'] = stock_info['종목코드'].apply(lambda x: '0' + str(x) + '.KS' if len(str(x)) < 6 else str(x) + '.KS')

    stock_dict = dict(zip(stock_info['종목명'], stock_info[['종목코드', '공모가(D)', '상장일_종가']].values))

    return stock_dict

def measure_downside_risk(start="2023-01-01", end="2023-12-31", positive_only=False):
    # Initialize an empty DataFrame and list for price changes
    all_stocks_data = pd.DataFrame()
    price_changes = []

    # Retrieve stock data
    stocks_dict = prepare(start, end)

    # Download historical data and compute downside risk
    for stock_name, (stock_symbol, ipo_price, initial_price) in stocks_dict.items():
        try:
            stock_data = yf.download(stock_symbol, start=start, end=end, progress=False)  # Disable progress bar
            stock_data['Stock Name'] = stock_name
            all_stocks_data = pd.concat([all_stocks_data, stock_data])

            # Calculate prices at the start, one month later, and three months later
            first_date = stock_data.index[0]
            one_month_later = first_date + timedelta(days=30)
            three_months_later = first_date + timedelta(days=90)

            first_price = stock_data.loc[first_date]['Close']
            one_month_later_price = stock_data.loc[stock_data.index >= one_month_later].iloc[0]['Close'] if stock_data.index[-1] >= one_month_later else None
            three_months_later_price = stock_data.loc[stock_data.index >= three_months_later].iloc[0]['Close'] if stock_data.index[-1] >= three_months_later else None

            if one_month_later_price and three_months_later_price:
                one_month_change = ((one_month_later_price - first_price) / first_price) * 100
                three_months_change = ((three_months_later_price - first_price) / first_price) * 100

                # Calculate the difference between IPO price and initial price
                ipo_initial_diff = initial_price - ipo_price

                # Filter stocks based on positive_only parameter
                if positive_only and (one_month_change < 0 or three_months_change < 0):
                    continue  # Skip stocks with negative changes if positive_only is set to True

                # Append the data for the stock
                price_changes.append({
                    'Stock': stock_name,
                    'IPO Price': ipo_price,
                    'Initial Price': initial_price,
                    'IPO-Initial Diff': ipo_initial_diff,
                    'First Price': first_price,
                    '1 Month Later Price': one_month_later_price,
                    '1 Month Change (%)': one_month_change,
                    '3 Months Later Price': three_months_later_price,
                    '3 Months Change (%)': three_months_change
                })

        except Exception as e:
            print(f"Error downloading data for {stock_name} ({stock_symbol}): {e}")

    # Convert the results into a DataFrame
    price_changes_df = pd.DataFrame(price_changes)

    # Save the DataFrame to an Excel file with the appropriate name based on the positive_only flag
    file_name = "downside risk analysis_positive only.xlsx" if positive_only else "downside risk analysis_negative included.xlsx"
    price_changes_df.to_excel(file_name, index=False)

### Downside Risk Measurement

In [ ]:
measure_downside_risk(start="2023-01-01", end="2023-12-31", positive_only=False)

In [ ]:
measure_downside_risk(start="2023-01-01", end="2023-12-31", positive_only=True)